In [1]:
from fastai import *
from fastai.text import *

In [2]:
import sklearn.feature_extraction.text as sklearn_text

In [3]:
?? URLs

In [4]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/Users/abhinavverma/.fastai/data/imdb_sample')

In [5]:
?? untar_data

In [6]:
df = pd.read_csv(path/'texts.csv')
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [7]:
movie_reviews = (TextList.from_csv(path, 'texts.csv', cols='text')
                         .split_from_df(col=2)
                         .label_from_df(cols=0))

In [13]:
len(movie_reviews.train.x), len(movie_reviews.valid.x)

(800, 200)

In [15]:
len(movie_reviews.vocab.itos), len(movie_reviews.vocab.stoi)

(6008, 19161)

In [17]:
movie_reviews.vocab.itos[100],movie_reviews.vocab.stoi["people"]

('people', 100)

In [18]:
#Create a countvectorizer
from collections import Counter
count_vector_train = Counter((movie_reviews.train.x)[0].data)

In [20]:
count_vector_valid = Counter((movie_reviews.valid.x)[0].data)

In [22]:
count_vector_train

Counter({2: 1,
         5: 15,
         4619: 1,
         25: 3,
         0: 8,
         867: 1,
         52: 5,
         3776: 1,
         1800: 1,
         95: 1,
         37: 1,
         85: 1,
         191: 1,
         64: 2,
         935: 1,
         2738: 1,
         517: 1,
         18: 1,
         21: 3,
         11: 1,
         84: 1,
         2417: 1,
         192: 1,
         88: 1,
         3777: 1,
         1801: 1,
         127: 1,
         10: 3,
         269: 1,
         15: 1,
         273: 1,
         73: 1,
         26: 2,
         9: 2,
         1360: 1,
         35: 2,
         1213: 1,
         1144: 1,
         1145: 1,
         2418: 1,
         91: 1,
         63: 1,
         245: 1,
         14: 2,
         1361: 1,
         1447: 1,
         65: 1,
         40: 1,
         796: 1,
         103: 1,
         72: 2,
         99: 1,
         534: 1,
         616: 1,
         48: 1,
         282: 1,
         54: 1,
         90: 1,
         219: 1,
         228: 1,

In [23]:
movie_reviews.valid.x[0].data

array([ 2,  5, 21, 71, ..., 15,  5,  0, 52])

In [28]:
indices = []

In [29]:
indices.extend(Counter(movie_reviews.valid.x[0].data))

In [30]:
indices

[2,
 5,
 21,
 71,
 189,
 748,
 288,
 285,
 63,
 221,
 666,
 59,
 13,
 2705,
 14,
 2875,
 11,
 18,
 358,
 0,
 77,
 15,
 478,
 1833,
 50,
 9,
 319,
 6,
 2743,
 12,
 115,
 4126,
 197,
 1331,
 25,
 324,
 10,
 3963,
 16,
 74,
 24,
 2817,
 5821,
 2595,
 710,
 3429,
 84,
 149,
 20,
 26,
 605,
 378,
 1057,
 251,
 258,
 1346,
 194,
 239,
 49,
 2764,
 1335,
 409,
 27,
 45,
 594,
 850,
 109,
 2601,
 430,
 1902,
 541,
 54,
 1107,
 608,
 404,
 736,
 44,
 204,
 23,
 3480,
 4736,
 456,
 4051,
 2420,
 30,
 337,
 966,
 58,
 207,
 2110,
 571,
 5035,
 579,
 1843,
 52]

In [31]:
def get_term_doc_matrix(label_list, vocab_len):
    j_indices = []
    indptr = []
    values = []
    indptr.append(0)

    for i, doc in enumerate(label_list):
        feature_counter = Counter(doc.data)
        j_indices.extend(feature_counter.keys())
        values.extend(feature_counter.values())
        indptr.append(len(j_indices))
        
#     return (values, j_indices, indptr)

    return scipy.sparse.csr_matrix((values, j_indices, indptr),
                                   shape=(len(indptr) - 1, vocab_len),
                                   dtype=int)


In [32]:
%%time
val_term_doc = get_term_doc_matrix(movie_reviews.valid.x, len(movie_reviews.vocab.itos))

CPU times: user 67.6 ms, sys: 21 ms, total: 88.6 ms
Wall time: 95.9 ms


In [33]:
%%time
trn_term_doc = get_term_doc_matrix(movie_reviews.train.x, len(movie_reviews.vocab.itos))

CPU times: user 178 ms, sys: 19.2 ms, total: 197 ms
Wall time: 227 ms


In [34]:
trn_term_doc.shape

(800, 6008)